In [18]:
from sklearn.linear_model import LinearRegression
from nba_api.stats.static import players
import pandas as pd
import numpy as np

In [19]:
all_players = players.get_players()
player_id_map = { p['full_name']: p['id'] for p in all_players }

In [39]:
df = pd.read_csv('lineup_plus_minus.csv')
df.describe()

,P1H,P2H,P3H,P4H,P5H,P1V,P2V,P3V,P4V,P5V,Plus_Off,Minus_Def,Plus/Minus,Home_Poss_Off,Home_Poss_Def,Poss_Tot,Time,Off_Rating,Def_Rating,Net_Rating
count,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,1.640000e+02,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,164.000000,142.000000,150.000000,134.000000
mean,1.307954e+06,1.314753e+06,1.462663e+06,1.358999e+06,1.396773e+06,1.297098e+06,1.316533e+06,1.444522e+06,1.400548e+06,1.483242e+06,3.414634,3.518293,-0.103659,2.981707,2.981707,5.963415,878.048780,105.567902,116.866897,-12.980540
std,5.981701e+05,5.975118e+05,4.653452e+05,5.664493e+05,5.314284e+05,6.119048e+05,5.954477e+05,4.889465e+05,5.354331e+05,4.408950e+05,4.482964,3.983821,3.155711,2.898006,2.874625,5.713927,844.061172,94.212031,102.702263,140.359922
min,2.011440e+05,2.544000e+03,2.544000e+03,2.544000e+03,2.011430e+05,2.544000e+03,2.544000e+03,2.544000e+03,2.544000e+03,2.544000e+03,0.000000,0.000000,-8.000000,0.000000,0.000000,0.000000,11.000000,0.000000,0.000000,-400.000000
25%,1.626164e+06,1.626167e+06,1.628970e+06,1.627832e+06,1.628384e+06,1.626164e+06,1.626157e+06,1.627775e+06,1.627820e+06,1.629000e+06,0.000000,0.000000,-2.000000,1.000000,1.000000,2.000000,337.500000,0.000000,0.000000,-98.750000
50%,1.628978e+06,1.628984e+06,1.630163e+06,1.630162e+06,1.629637e+06,1.629626e+06,1.629086e+06,1.630162e+06,1.630174e+06,1.630541e+06,2.000000,2.500000,0.000000,2.000000,2.000000,5.000000,655.000000,100.000000,100.000000,0.000000
75%,1.630578e+06,1.630306e+06,1.630559e+06,1.630595e+06,1.631095e+06,1.630595e+06,1.630614e+06,1.630595e+06,1.631095e+06,1.631202e+06,5.000000,6.000000,2.000000,4.000000,4.000000,7.000000,1100.000000,150.000000,200.000000,66.666667
max,1.642261e+06,1.642261e+06,1.642359e+06,1.642402e+06,1.642402e+06,1.642402e+06,1.642402e+06,1.642402e+06,1.642402e+06,1.642402e+06,32.000000,27.000000,8.000000,18.000000,19.000000,37.000000,5890.000000,500.000000,400.000000,400.000000


In [70]:
all_slots = ["P1H","P2H","P3H","P4H","P5H","P1V","P2V","P3V","P4V","P5V"]
players = pd.unique(df[all_slots].values.ravel())

X_DEF = pd.DataFrame(0, index=df.index, columns = players)
X_OFF = pd.DataFrame(0, index=df.index, columns = players)

for i, row in df.iterrows():
    if not np.isnan(row['Off_Rating']):
        for slot in ["P1H","P2H","P3H","P4H","P5H"]:
            player_id = row[slot]
            X_OFF.at[i, player_id] = 1
        for slot in ["P1V","P2V","P3V","P4V","P5V"]:
            player_id = row[slot]
            X_OFF.at[i, player_id] = -1

    if not np.isnan(row['Def_Rating']):
        for slot in ["P1H","P2H","P3H","P4H","P5H"]:
            player_id = row[slot]
            X_DEF.at[i, player_id] = 1
        for slot in ["P1V","P2V","P3V","P4V","P5V"]:
            player_id = row[slot]
            X_DEF.at[i, player_id] = -1

In [76]:
X_DEF = X_DEF.loc[(X_DEF != 0).any(axis=1), :]
X_OFF = X_OFF.loc[(X_OFF != 0).any(axis=1), :]   

In [77]:
X_DEF.describe()

,1626157,1628384,1628404,1628969,1628973,201143,201950,1627759,1628369,1628401,...,1630191,1631323,204456,1630167,1631097,1631204,1641842,1641716,1642402,1641767
count,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,...,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000,150.000000
mean,0.006667,-0.006667,0.040000,0.046667,0.040000,-0.033333,-0.046667,-0.020000,0.006667,-0.006667,...,-0.013333,0.000000,0.026667,0.033333,0.026667,-0.020000,0.000000,-0.040000,-0.026667,-0.013333
std,0.183064,0.271626,0.196616,0.267644,0.196616,0.214153,0.241268,0.244949,0.245679,0.216645,...,0.347313,0.327693,0.326599,0.336118,0.400447,0.140469,0.327693,0.228212,0.230164,0.200224
min,-1.000000,-1.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000


In [78]:
y_DEF = df.loc[X_DEF.index, 'Def_Rating']
y_OFF = df.loc[X_OFF.index, 'Off_Rating']

In [82]:
model_def = LinearRegression()
model_def.fit(X_DEF, y_DEF)
coeficients = pd.Series(model_def.coef_, index=X_DEF.columns)
intercept = pd.Series(model_def.intercept_)

model_off = LinearRegression()
model_off.fit(X_OFF, y_OFF)
coeficients = pd.Series(model_off.coef_, index=X_OFF.columns)
intercept = pd.Series(model_off.intercept_)

In [81]:
coeficients

1626157     5.881964
1628384   -19.118036
1628404    -1.028721
1628969   -65.112650
1628973    -1.028721
             ...    
1631204    37.752593
1641842   -71.300069
1641716   -37.404487
1642402    85.897092
1641767    24.281934
Length: 110, dtype: float64